# Test set performance

In [87]:
# Define classes for preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import FeatureUnion
from pandas.tseries.offsets import DateOffset

# select specific columns for pipeline
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

# preprocessing:
# - merge in store data
# - add additional columns
# - remove unneeded rows and columns
class AttributeAdder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.storefile = pd.read_csv('data/store.csv')
    def fit(self, X, y=None):
        X['Date'] = pd.to_datetime(X['Date'], errors='coerce')
        last_date = max(X['Date'])
        interval_start = last_date - DateOffset(months=3)
        self.stores = X.loc[(X['Store'] > 0) & (X['Sales'] > 0) & (X['Date'] > interval_start), 
                            ['Store', 'Sales']].groupby(['Store'])
        
        return self
    def transform(self, X, y=None):     
        
        # remove Sales =0 
        X = X[X['Sales'] > 0]
        
        # remove Stores without id
        X = X[X['Store'] > 0]
    
        
        # merge in store data
        X = pd.merge(X, self.storefile, how='left', on='Store')
        
        # create features mean and median of Sales per store over last 3 months
 
        for store, v in self.stores['Sales']:
            X.loc[X['Store'] == store, 'Store_mean'] = v.mean()
            X.loc[X['Store'] == store, 'Store_median'] = v.median()
       
        # Create more date columns
        X['Date'] = pd.to_datetime(X['Date'], errors='coerce')
        X['Year'] = X.Date.dt.year
        X['Month'] = X.Date.dt.month
        X['DayOfWeek'] = X.Date.dt.dayofweek
        
        # Create feature Competition since months, cap for high values
        X["CompetitionSinceMonths"] = ((X['Year']- X['CompetitionOpenSinceYear']) * 12 +
                                           (X['Month'] - X['CompetitionOpenSinceMonth']))
        X.loc[X["CompetitionSinceMonths"] < 0, "CompetitionSinceMonths"] = 0
        X.loc[X["CompetitionSinceMonths"] > 24, "CompetitionSinceMonths"] = 24
        
        # Set competition distance at cap if competition is not yet open
        max_distance = 10000.
        X.loc[X["CompetitionSinceMonths"] == 0, 'CompetitionDistance'] = max_distance
        X.loc[X['CompetitionDistance'] > max_distance, 'CompetitionDistance']  = max_distance
        
        # Create promo since column, cap for high values
        X["Promo2SinceWeeks"] = ((X['Year']- X['Promo2SinceYear']) * 52 +
                                           (X['Month'] - X['Promo2SinceWeek']))
        X.loc[X["Promo2SinceWeeks"] < 0, "Promo2SinceWeeks"] = 0
        X.loc[X["Promo2SinceWeeks"] > 12, "Promo2SinceWeeks"] = 12
        
        # Clean up state holiday
        X['StateHoliday'] = X['StateHoliday'].replace(0.0, "0")
        X['StateHoliday'] = X['StateHoliday'].replace("0", 0)
        X['StateHoliday'] = X['StateHoliday'].replace("a", "Public")
        X['StateHoliday'] = X['StateHoliday'].replace("b", "Easter")
        X['StateHoliday'] = X['StateHoliday'].replace("c", "Christmas")
        
        # drop columns
        X = X.drop(columns = ['Customers', 'Open', "CompetitionOpenSinceMonth",
                             'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear',
                             'PromoInterval', 'Date']
        )
        self.attribute_names = X.columns
        return X


# define evaluation metrics RMSPE
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])


# full pipeline including attribute adding, missing value imputing
# one-hot encoding
def full_pipeline(data, fit=True):
    
    if fit:
        transformed = pp.fit_transform(data)
    else:
        transformed = pp.transform(data) 
    
    impute_miss_cat = ['SchoolHoliday', 'StateHoliday', "Promo"]
    si = SimpleImputer(strategy="constant", fill_value = 0)
    
    dfs = DataFrameSelector(impute_miss_cat)
    t = dfs.fit_transform(transformed)
    cat_tr = si.fit_transform(t)

    transformed.loc[:, impute_miss_cat] = cat_tr
    
    numerical_features = ['CompetitionDistance', "CompetitionSinceMonths", "Promo2SinceWeeks"]
                    
    target = ['Sales']
    num_columns = numerical_features + target
    num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_columns)),
        ('imputer', SimpleImputer(strategy="median")),
       
    ])
    
    num_tr = num_pipeline.fit_transform(transformed)
    transformed.loc[:, num_columns] = num_tr        
    
    one_hot_cat = ['StateHoliday', 'StoreType', 'Assortment',
                   'Month', 'DayOfWeek']
    transformed = pd.get_dummies(transformed, columns=one_hot_cat)
    return transformed

In [88]:
# Load data
import pandas as pd
train_data = pd.read_csv("data/train.csv")
store = pd.read_csv('data/store.csv')


# Insert test data link here
test = pd.read_csv("https://raw.githubusercontent.com/igivis7/dsr28_minicomp_team1/main/data/holdout.csv")


In [90]:
# run preprocessing on training to fit some values
train_transformed = full_pipeline(train_data)

In [91]:
# run preprocessing on test set
test_trans = full_pipeline(test, fit=False)
for c in train_transformed.columns:
    if not c in test_trans.columns:
        test_trans[c] = 0


In [92]:
# columns to keep
keep_columns = ["Promo", "Promo2SinceWeeks", "CompetitionSinceMonths", 
                "Store_mean", "CompetitionDistance", "Month_12", "DayOfWeek_0", 
                "DayOfWeek_1", "DayOfWeek_5"]

X_train = train_transformed.loc[:, keep_columns]
y_train = train_transformed['Sales']
X_valid = test_trans.loc[:, keep_columns]
y_valid = test_trans['Sales']

In [95]:
from sklearn.tree import DecisionTreeRegressor
import pickle

tree_reg_trained = pickle.load(open("model/tree_26.5.sav", "rb"))
y_pred = tree_reg_trained.predict(X_valid)
rmspe = metric(y_pred, np.array(y_valid))
print(f'Prediction: decision tree, RMSPE={rmspe:.2f}%')

Prediction: decision tree, RMSPE=26.51%


# Data cleaning

In [35]:
pp = AttributeAdder()

In [39]:

train_transformed = full_pip(newtrain)
train_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 497376 entries, 0 to 497375
Data columns (total 41 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Store                   497376 non-null  float64
 1   Sales                   497376 non-null  float64
 2   Promo                   497376 non-null  float64
 3   SchoolHoliday           497376 non-null  float64
 4   CompetitionDistance     497376 non-null  float64
 5   Promo2                  497376 non-null  int64  
 6   Store_mean              497376 non-null  float64
 7   Store_median            497376 non-null  float64
 8   Year                    497376 non-null  int64  
 9   CompetitionSinceMonths  497376 non-null  float64
 10  Promo2SinceWeeks        497376 non-null  float64
 11  StateHoliday_0          497376 non-null  uint8  
 12  StateHoliday_Christmas  497376 non-null  uint8  
 13  StateHoliday_Easter     497376 non-null  uint8  
 14  StateHoliday_Public 

In [954]:
train_transformed.to_csv("data/fulltrain_transformed.csv", header=True)

In [40]:
valid_trans = full_pip(validation, fit=False)
for c in train_transformed.columns:
    if not c in valid_trans.columns:
        valid_trans[c] = 0

In [956]:
valid_trans.to_csv("data/holdout_transformed.csv", header=True)

# Stationary model

In [75]:
# Linear regression model
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
y_pred = lin_reg.predict(X_valid)
rmspe = metric(y_pred, np.array(y_valid))

print(f'Prediction: linear regression, RMSPE={rmspe:.2f}%')
y_pred_train = lin_reg.predict(X_train)
rmspe = metric(y_pred_train, np.array(y_train))
print(f'On training set, RMSPE={rmspe:.2f}%')

Prediction: linear regression, RMSPE=32.42%
On training set, RMSPE=29.76%


In [82]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)
y_pred = tree_reg.predict(X_valid)
rmspe = metric(y_pred, np.array(y_valid))
print(f'Prediction: decision tree, RMSPE={rmspe:.2f}%')

Prediction: decision tree, RMSPE=26.51%


In [83]:
import pickle
filename = 'tree_26.5.sav'
pickle.dump(tree_reg, open(filename, 'wb'))


In [65]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.)
gbrt.fit(X_train, y_train)
y_pred = gbrt.predict(X_valid)
rmspe = metric(y_pred, np.array(y_valid))
print(f'Gradient boosting Reg, RMSPE={rmspe:.2f}%')

Gradient boosting Reg, RMSPE=34.09%


In [56]:
import xgboost as xgb
from xgboost import plot_importance

dtrain = xgb.DMatrix(X_train, label=y_train)
deval = xgb.DMatrix(X_valid, label=y_valid)

params_upd1 = {"objective": "reg:squarederror", #since it is a regression problem
          "booster" : "gbtree",     #tree
          "eta": 0.1,              #learning rate   to reduce overfitting issues
    "max_depth": 10,          #depth of the tree
    #      "eta": 1.0,     
    #    "max_depth": 2,
          "subsample": 0.9,         #subsample the data prior to growing trees - overcomes overfitting
          "colsample_bytree": 0.7,  #subsampling of columns for each tree
          "seed": 10                
          }
num_of_trees = 30
num_of_trees = 100
early_stopping_rounds_N=20

xgb_model_MK2 = xgb.train(
                          params_upd1, 
                          dtrain, 
                          num_boost_round=num_of_trees,
                          early_stopping_rounds=early_stopping_rounds_N, 
                          evals=[(deval, "Eval")], 
                          verbose_eval=False
                         )

y_pred = xgb_model_MK2.predict(deval)
rmspe = metric(y_pred, np.array(y_valid))
print(f'XGBoost, RMSPE={rmspe:.2f}%')

XGBoost, RMSPE=27.34%


In [58]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor(max_depth=20)
forest_reg.fit(X_train, y_train)
y_pred = forest_reg.predict(X_valid)
rmspe = metric(y_pred, np.array(y_valid))
print(f'Prediction: random forest, RMSPE={rmspe:.2f}%')

Prediction: random forest, RMSPE=26.07%


In [61]:
#plt.barh(X_train.columns, forest_reg.feature_importances_)
df = pd.DataFrame({
    "col": X_train.columns,
    "importance" : forest_reg.feature_importances_,   
})
# Plotly Express is now a sublibrary of Plotly:
import plotly.express as px
fig = px.scatter(
    data_frame=df.sort_values(by='importance'),
    x='col',
    y='importance',
    title='Feature importance',
    log_y = True
)
fig.add_hline(y=0.02)
fig.show()

In [ ]:
#from sklearn.model_selection import GridSearchCV
#param_grid = [
#    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    
    
#]


In [718]:
# Check stationarity

from statsmodels.tsa.stattools import adfuller
pvalues = []
store_ids = newtrain_set['Store'].unique()
for id in store_ids:
    result = adfuller(newtrain_set.loc[newtrain_set['Store'] == id, 'Sales'])
 #   print('ADF Statistic: %f' % result[0])
    pvalues.append(result[1])

df_pvalues = pd.DataFrame({
        "Store" : store_ids,
        "pvalue" : pvalues,
})

# Plotly Express is now a sublibrary of Plotly:
import plotly.express as px
fig = px.scatter(
    data_frame=df_pvalues,
    x='Store',
    y='pvalue',
    title='P values per store',
    log_y = True
)
fig.add_hline(y=0.05)
fig.show()




KeyboardInterrupt: 